In [1]:
!pip install flask flask-sqlalchemy flask-login validators pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.6 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
ngrok.set_auth_token("3849aq7oMilBPYk1KGOmmmgnpHq_4twiCVrGpivRREYwExE5W")


In [3]:
import os
os.makedirs("/content/templates", exist_ok=True)
os.makedirs("/content/static", exist_ok=True)


In [4]:
%%writefile /content/app.py
from flask import Flask, render_template, request, redirect, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
import string, random, validators
from pyngrok import ngrok

app = Flask(__name__, template_folder="/content/templates")
app.secret_key = "secret123"

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///database.db"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = SQLAlchemy(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"

# ---------------- DATABASE MODELS ----------------

class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(9), unique=True, nullable=False)
    password = db.Column(db.String(100), nullable=False)

class URL(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    original_url = db.Column(db.String(500), nullable=False)
    short_code = db.Column(db.String(6), nullable=False)
    user_id = db.Column(db.Integer, db.ForeignKey("user.id"))

# ---------------- LOGIN LOADER ----------------

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# ---------------- UTILITY ----------------

def generate_short_code(length=6):
    chars = string.ascii_letters + string.digits
    return ''.join(random.choice(chars) for _ in range(length))

# ---------------- ROUTES ----------------

@app.route("/", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        user = User.query.filter_by(username=request.form["username"]).first()
        if user and user.password == request.form["password"]:
            login_user(user)
            return redirect("/dashboard")
        flash("Invalid username or password")
    return render_template("login.html")

@app.route("/signup", methods=["GET", "POST"])
def signup():
    if request.method == "POST":
        username = request.form["username"]
        password = request.form["password"]

        if len(username) < 5 or len(username) > 9:
            flash("Username must be between 5 to 9 characters long")
            return redirect("/signup")

        if User.query.filter_by(username=username).first():
            flash("This username already exists")
            return redirect("/signup")

        db.session.add(User(username=username, password=password))
        db.session.commit()
        flash("Signup successful. Please login.")
        return redirect("/")
    return render_template("signup.html")

@app.route("/dashboard", methods=["GET", "POST"])
@login_required
def dashboard():
    short_url = None
    if request.method == "POST":
        original_url = request.form["original_url"]
        if validators.url(original_url):
            code = generate_short_code()
            db.session.add(URL(
                original_url=original_url,
                short_code=code,
                user_id=current_user.id
            ))
            db.session.commit()
            short_url = request.host_url + code
    urls = URL.query.filter_by(user_id=current_user.id).all()
    return render_template("dashboard.html", short_url=short_url, urls=urls)

@app.route("/<code>")
def redirect_url(code):
    url = URL.query.filter_by(short_code=code).first_or_404()
    return redirect(url.original_url)

@app.route("/logout")
@login_required
def logout():
    logout_user()
    return redirect("/")

# ---------------- RUN ----------------

if __name__ == "__main__":
    with app.app_context():
        db.create_all()

    public_url = ngrok.connect(5000)
    print("PUBLIC URL:", public_url)

    app.run(host="0.0.0.0", port=5000)


Writing /content/app.py


In [5]:
%%writefile /content/templates/login.html
<!DOCTYPE html>
<html>
<head>
    <title>Login</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="container mt-5">

<h3>Login</h3>

{% with messages = get_flashed_messages() %}
{% for msg in messages %}
<p class="text-danger">{{ msg }}</p>
{% endfor %}
{% endwith %}

<form method="POST">
    <input type="text" name="username" class="form-control" placeholder="Username" required>
    <input type="password" name="password" class="form-control mt-2" placeholder="Password" required>
    <button class="btn btn-primary mt-3">Login</button>
</form>

<p class="mt-3">
    Don't have an account? <a href="/signup">Signup</a>
</p>

</body>
</html>


Writing /content/templates/login.html


In [6]:
%%writefile /content/templates/signup.html
<!DOCTYPE html>
<html>
<head>
    <title>Signup</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="container mt-5">

<h3>Signup</h3>

{% with messages = get_flashed_messages() %}
{% for msg in messages %}
<p class="text-danger">{{ msg }}</p>
{% endfor %}
{% endwith %}

<form method="POST">
    <input type="text" name="username" class="form-control" placeholder="Username (5–9 chars)" required>
    <input type="password" name="password" class="form-control mt-2" placeholder="Password" required>
    <button class="btn btn-success mt-3">Signup</button>
</form>

<p class="mt-3">
    Already have an account? <a href="/">Login</a>
</p>

</body>
</html>


Writing /content/templates/signup.html


In [7]:
%%writefile /content/templates/dashboard.html
<!DOCTYPE html>
<html>
<head>
    <title>Dashboard</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="container mt-5">

<h3>URL Shortener</h3>
<a href="/logout" class="btn btn-danger btn-sm">Logout</a>

<form method="POST" class="mt-4">
    <input type="text" name="original_url" class="form-control" placeholder="Enter URL" required>
    <button class="btn btn-primary mt-2">Shorten URL</button>
</form>

{% if short_url %}
<input class="form-control mt-3" value="{{ short_url }}" readonly>
<button class="btn btn-success mt-2"
onclick="navigator.clipboard.writeText('{{ short_url }}')">
Copy
</button>
{% endif %}

<h5 class="mt-4">Your URL History</h5>
<ul class="list-group">
{% for url in urls %}
<li class="list-group-item">
{{ request.host_url }}{{ url.short_code }} → {{ url.original_url }}
</li>
{% endfor %}
</ul>

</body>
</html>


Writing /content/templates/dashboard.html


In [ ]:
!python /content/app.py


PUBLIC URL: NgrokTunnel: "https://directional-superaffluently-ayako.ngrok-free.dev" -> "http://localhost:5000"
 * Serving Flask app 'app'
 * Debug mode: off
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Jan/2026 12:03:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2026 12:03:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Jan/2026 12:04:38] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2026 12:04:47] "GET /signup HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2026 12:05:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2026 12:06:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2026 12:08:01] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2026 12:08:29] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2026 12:08:34] "GET /signup HTTP/1.1" 200 -
127.0.0.1 - - [22/Jan/2026 12:08:53] "POST /signup HTTP/1.1" 302 -
127.0.0.1 - - [22/Jan/2026 12:08:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22